In [1]:
import tensorflow as tf
import numpy as np
import time
from datetime import timedelta

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets('data/MNIST/', one_hot=True)
print("Size of:")
print("- Training-set:\t\t{}".format(len(data.train.labels)))
print("- Test-set:\t\t{}".format(len(data.test.labels)))
print("- Validation-set:\t{}".format(len(data.validation.labels)))


Extracting data/MNIST/train-images-idx3-ubyte.gz
Extracting data/MNIST/train-labels-idx1-ubyte.gz
Extracting data/MNIST/t10k-images-idx3-ubyte.gz
Extracting data/MNIST/t10k-labels-idx1-ubyte.gz
Size of:
- Training-set:		55000
- Test-set:		10000
- Validation-set:	5000


In [3]:
img_size = 28
img_size_flat = img_size * img_size
img_shape = (img_size, img_size)
num_channels = 1
num_classes = 10

# Convolutional Layer 1.
filter_size1 = 5          # Convolution filters are 5 x 5 pixels.
num_filters1 = 16         # There are 16 of these filters.

# Convolutional Layer 2.
filter_size2 = 5          # Convolution filters are 5 x 5 pixels.
num_filters2 = 36         # There are 36 of these filters.

# Fully-connected layer.
fc_size = 128             # Number of neurons in fully-connected layer.

In [20]:
tf.reset_default_graph()
def conv_layer(name, input, filter_size, in_filter, out_filter):
    with tf.variable_scope(name):
        weights = tf.get_variable(name='dw', shape=[filter_size, filter_size, in_filter, out_filter], initializer=tf.truncated_normal_initializer(stddev=0.05))
        bias = tf.get_variable(name='bw', shape=[out_filter], initializer=tf.constant_initializer(0.05))
        layer = tf.nn.conv2d(input=input, filter=weights, strides=[1, 1, 1, 1], padding='SAME')
        layer += bias
        layer = tf.nn.relu(layer)
        layer = tf.nn.max_pool(value=layer, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        return layer

def flat_layer(layer):
    layer_shape = layer.get_shape()
    # The shape of the input layer is assumed to be:
    # layer_shape == [num_images, img_height, img_width, num_channels]
    num_features = layer_shape[1:4].num_elements()
    layer_flat = tf.reshape(layer, [-1, num_features])
    return layer_flat, num_features

def full_conn(input, input_dim, out_dim):
    weights = tf.get_variable(name='dw', shape=[input_dim, out_dim], initializer=tf.truncated_normal_initializer(stddev=0.05))
    bias = tf.get_variable('bw', shape=[out_dim], initializer=tf.constant_initializer(0.05))
    return tf.nn.xw_plus_b(input, weights, bias)

In [21]:
x = tf.placeholder(tf.float32, shape=[None, img_size_flat], name='x')
x_image = tf.reshape(x, [-1, img_size, img_size, num_channels])
y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')
y_true_cls = tf.argmax(y_true, dimension=1)

In [22]:
layer_conv1 = conv_layer('conv1', input=x_image, filter_size=filter_size1, in_filter=num_channels, out_filter=num_filters1)
layer_conv2 = conv_layer('conv2', input=layer_conv1, filter_size=filter_size2, in_filter=num_filters1, out_filter=num_filters2)
layer_flat, num_features = flat_layer(layer_conv2)
layer_fc = full_conn(input=layer_flat, input_dim=num_features, out_dim=num_classes)

In [23]:
y_pred = tf.nn.softmax(layer_fc)
y_pred_cls = tf.argmax(y_pred, dimension=1)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=layer_fc, labels=y_true)
cost = tf.reduce_mean(cross_entropy)

In [26]:
variables = tf.trainable_variables()
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4)
correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
grads = tf.gradients(cost, variables)
pred = tf.nn.softmax(layer_fc)
apply_grad = optimizer.apply_gradients(zip(grads, variables))

In [9]:
grad1 = tf.gradients(pred[0], )

In [27]:
session = tf.Session()
session.run(tf.global_variables_initializer())

In [30]:
train_batch_size = 64
total_iterations = 0

def optimize(num_iterations):
    # Ensure we update the global variable rather than a local copy.
    global total_iterations

    # Start-time used for printing time-usage below.
    start_time = time.time()

    for i in range(total_iterations,
                   total_iterations + num_iterations):
        x_batch, y_true_batch = data.train.next_batch(train_batch_size)
        feed_dict_train = {x: x_batch,
                           y_true: y_true_batch}
        opt_ = session.run([apply_grad], feed_dict=feed_dict_train)

        # Calculate the accuracy on the training-set.
        acc = session.run(accuracy, feed_dict=feed_dict_train)
        # Message for printing.
        msg = "Optimization Iteration: {0:>6}, Training Accuracy: {1:>6.1%}"
        print(msg.format(i + 1, acc))
    total_iterations += num_iterations
    
    end_time = time.time()
    # Difference between start and end-times.
    time_dif = end_time - start_time

    # Print the time-usage.
    print("Time usage: " + str(timedelta(seconds=int(round(time_dif)))))

In [31]:
optimize(num_iterations=50)

Optimization Iteration:      1, Training Accuracy:  17.2%
Optimization Iteration:      2, Training Accuracy:  14.1%
Optimization Iteration:      3, Training Accuracy:  12.5%
Optimization Iteration:      4, Training Accuracy:  15.6%
Optimization Iteration:      5, Training Accuracy:  21.9%
Optimization Iteration:      6, Training Accuracy:  20.3%
Optimization Iteration:      7, Training Accuracy:  23.4%
Optimization Iteration:      8, Training Accuracy:  14.1%
Optimization Iteration:      9, Training Accuracy:  17.2%
Optimization Iteration:     10, Training Accuracy:  25.0%
Optimization Iteration:     11, Training Accuracy:  21.9%
Optimization Iteration:     12, Training Accuracy:  23.4%
Optimization Iteration:     13, Training Accuracy:  20.3%
Optimization Iteration:     14, Training Accuracy:  29.7%
Optimization Iteration:     15, Training Accuracy:  28.1%
Optimization Iteration:     16, Training Accuracy:  43.8%
Optimization Iteration:     17, Training Accuracy:  32.8%
Optimization I

(5, 5, 1, 16)

In [40]:
x_batch, y_true_batch = data.train.next_batch(train_batch_size)
feed_dict_train = {x: x_batch, y_true: y_true_batch, variables[0]: np.random.rand(5,5,1,16)}
acc_, vars_, grads = session.run([accuracy, variables, grads], feed_dict=feed_dict_train)

In [41]:
grads

[array([[[[-0.01265985,  0.08633113,  0.05567193,  0.12586455,  0.0848004 ,
            0.02237625,  0.01909425, -0.10680807,  0.08045077, -0.0964698 ,
            0.1122604 , -0.01628481,  0.08031808,  0.1123568 ,  0.10100253,
           -0.14798409]],
 
         [[-0.01722285,  0.10886842,  0.08526801,  0.12024938,  0.06887244,
            0.04346335,  0.02581882, -0.1109385 ,  0.11054616, -0.1024368 ,
            0.0545208 , -0.0335637 ,  0.08544011,  0.13426428,  0.12414464,
           -0.16100945]],
 
         [[ 0.01138775,  0.1338719 ,  0.08225578,  0.141367  ,  0.06585848,
            0.05189118,  0.02938674, -0.08286104,  0.08746118, -0.09022603,
            0.02672992, -0.07173009,  0.09159033,  0.11735144,  0.14514211,
           -0.15838008]],
 
         [[ 0.02215786,  0.15657385,  0.07640894,  0.12585126,  0.08800127,
            0.04466159,  0.0484971 , -0.0322585 ,  0.07813358, -0.06818259,
            0.0196992 , -0.06903397,  0.08559009,  0.0772436 ,  0.12097379,
    

In [138]:
var_input = tf.placeholder(tf.float32, shape=[5,5,1,16])
ass_op = tf.assign(variables[0], var_input)

In [39]:
feed_dict_test = {var_input: np.random.rand(5,5,1,16)}
session.run(grad, feed_dict=feed_dict_test)

NameError: name 'var_input' is not defined

In [37]:
vars_ = session.run(variables)
np.sum(list(map(lambda x: np.linalg.norm(x) ** 2, vars_)))

63.48924471318373

In [38]:
session.run(grad)

InvalidArgumentError: Shape [-1,784] has negative dimensions
	 [[Node: x = Placeholder[dtype=DT_FLOAT, shape=[?,784], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'x', defined at:
  File "/home/jczeng/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/jczeng/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/jczeng/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/jczeng/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/jczeng/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/home/jczeng/anaconda3/lib/python3.6/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "/home/jczeng/anaco

In [ ]:
a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2,3], name='a')
b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3,2], name='b')
c = tf.matmul(a, b)

In [103]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [5]:
sess.run(c)

array([[ 22.,  28.],
       [ 49.,  64.]], dtype=float32)

In [10]:
sess.run(tf.slice(a, [1, 0], [1, 1]))

array([[ 4.]], dtype=float32)

In [11]:
e = tf.constant([1,1,1,2,2,2,3,3,3,4,4,4,5,5,5,6,6,6], shape=[3,2,3], name='e')

In [23]:
sess.run(tf.slice(e, [1,1,1], [2,1,2]))

array([[[4, 4]],

       [[6, 6]]], dtype=int32)

In [31]:
sess.run(tf.strided_slice(e, [1,1,1], [3,2,3], [2,1,1]))

array([[[4, 4]]], dtype=int32)

In [113]:
a = tf.constant([0.01,3, 0.01], name='a')
b = tf.constant([0.0,1.0,0.0],name='b')
c = tf.nn.softmax_cross_entropy_with_logits(logits=a, labels=b)

In [107]:
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), 0)

In [112]:
softmax([0.01, 3, 0.01])

array([ 0.04569197,  0.90861606,  0.04569197])

In [114]:
sess.run(c)

0.095832743

In [2]:
m = np.array([0.11,0.1,1,2,2,2,3,3,3,4,4,4])
n = np.array([1,2,5,1,2,5,1,2,5,1,2,5])

In [63]:
t = np.reshape(m, [2,2,3])
f = np.reshape(n, [12,1])

In [12]:
ids1 = np.argwhere(m == 2).ravel()
ids2 = np.argwhere(m == 3).ravel()
np.concatenate((ids1, ids2))

array([3, 4, 5, 6, 7, 8])

In [75]:
i = np.argwhere(m==0.11).ravel()

In [91]:
# Creates a graph.
a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(c))

[[ 22.  28.]
 [ 49.  64.]]


In [119]:
with tf.variable_scope("foo"):
    with tf.variable_scope("bar"):
        m = tf.get_variable('dt1', [1])

In [109]:
m.name

'foo/dt:0'

In [123]:
with tf.variable_scope("foo", reuse=True):
    with tf.variable_scope("bar"):
        m2 = tf.get_variable("dt1", [1])

In [149]:
x = tf.placeholder(tf.float32, shape=[None, 5], name='x')
x1 = tf.placeholder(tf.float32, shape=[None, 5], name='x1')

In [150]:
x_batch = np.array([[1,2,3,4,5], [5,4,3,2,1]])

In [151]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
sess.run(x, feed_dict={x:x_batch})
print(x.get_shape())

array([[ 1.,  2.,  3.,  4.,  5.],
       [ 5.,  4.,  3.,  2.,  1.]], dtype=float32)

In [153]:
print(x.get_shape()[1])

5


In [17]:
arr = np.arange(9)
np.random.shuffle(arr)
arr.reshape((3,3))

array([[6, 1, 5],
       [4, 3, 8],
       [0, 2, 7]])

In [18]:
np.ones(4)*25

array([ 25.,  25.,  25.,  25.])

In [20]:
np.sum(np.array([12,21]))

33

In [26]:
np.unique(np.array([1,1,22]))

array([ 1, 22])

In [31]:
np.count_nonzero(m==3)


3

In [35]:
np.concatenate([np.array([]),[1,2,3]])

array([ 1.,  2.,  3.])

In [146]:
int(3.3)

3

In [147]:
c = [("a",2,3), ("2",1,4)]
np.random.choice([4,5,3,2], 0)

array([], dtype=int64)

In [70]:
import re
re.split(r'[\s+,]', "I like play , and you")

['I', 'like', 'play', '', '', 'and', 'you']

In [75]:
np.argwhere(m[[1,2,3]] > 1).ravel()

array([2])

In [86]:
m.reshape([3,2,2])

array([[[ 0.11,  0.1 ],
        [ 1.  ,  2.  ]],

       [[ 2.  ,  2.  ],
        [ 3.  ,  3.  ]],

       [[ 3.  ,  4.  ],
        [ 4.  ,  4.  ]]])

In [87]:
t = m.reshape([3,2,2])

In [88]:
np.max(t, 2)

array([[ 0.11,  2.  ],
       [ 2.  ,  3.  ],
       [ 4.  ,  4.  ]])

In [99]:
t[(2,0),(1,1)]

array([[ 4.,  4.],
       [ 1.,  2.]])

In [116]:
np.argsort(m)

array([ 1,  0,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [118]:
m

array([ 0.11,  0.1 ,  1.  ,  2.  ,  2.  ,  2.  ,  3.  ,  3.  ,  3.  ,
        4.  ,  4.  ,  4.  ])

In [119]:
set(m)- set([1,2,3])

{0.10000000000000001, 0.11, 4.0}

In [125]:
m[list(set([1,2,3]))]

array([ 0.1,  1. ,  2. ])

In [124]:
np.array(list(set([1,2,3])))

array([1, 2, 3])

In [136]:
m.reshape([3,4])

array([[ 0.11,  0.1 ,  1.  ,  2.  ],
       [ 2.  ,  2.  ,  3.  ,  3.  ],
       [ 3.  ,  4.  ,  4.  ,  4.  ]])

In [137]:
j = np.argsort(m.reshape([3,4]), axis=0).astype(np.int32)

In [164]:
j

array([[0, 0, 0, 0],
       [1, 1, 1, 1],
       [2, 2, 2, 2]], dtype=int32)

In [158]:
22 in [list(range(1,23)), list(range(1,23))]

False

In [163]:
2 in list(range(10, 40)) + list(range(50, 80)) + list(range(90, 120))

False

In [178]:
np.argsort(m.reshape([3,4]), axis=1)

array([[1, 0, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3]])

In [179]:
np.abs(m)

array([ 0.11,  0.1 ,  1.  ,  2.  ,  2.  ,  2.  ,  3.  ,  3.  ,  3.  ,
        4.  ,  4.  ,  4.  ])